In [1]:
from utils_misc import pickle_load_or_create, pickle_save

In [2]:
from DataRepository import DataRepository


cls = DataRepository()


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
connection established


In [3]:
cls.query_framelabels()



cls.get_path(2)

cls.get_randomized_borderlabels_and_batches_per_video(20)

,videoID,batches_in_video,random_order,total_batches
0,2,38,1,38
1,4,19,2,57
2,6,16,3,73
3,3,23,4,96
4,1,24,5,120
5,5,20,6,140


In [4]:
cls.get_borderlabels_batch(videoID=3, batch_nr=8, batch_size=20)

,frameNr,label
0,160,0
1,161,0
2,162,0
3,163,1
4,164,1
5,165,2
6,166,2
7,167,2
8,168,2
9,169,2


In [5]:
from utils_cv2 import get_frames, get_video_length

In [6]:
path = '../videos/20240201_atelier_003.mp4'

In [7]:
get_video_length(path)

1489

In [8]:
frames = get_frames('../videos/20240201_atelier_003.mp4', 1450, 1500, (150,150), rgb=True)
frames.shape

(51, 150, 150, 3)

In [9]:
frames[50][0], frames[22][0]

(array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0

In [24]:
from DataGeneratorBordersDB import DataGeneratorSkillBorders

gen = DataGeneratorSkillBorders(train=True, batch_size=20, dim=(128, 128), n_channels=3,
                 n_classes=5, shuffle=True, time_length=16)

connection established
DataGeneratorSkillBorders init done


In [25]:
gen

In [26]:
%%time
for i in range(22,60):
    print(i)
    X, y = gen.__getitem__(3)

22
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
23
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
24
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
25
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
26
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
27
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
28
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
29
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
30
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
31
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
32
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
33
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
34
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
35
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
36
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
37
did you call me? batch: 3 shape is: (20, 1, 128, 128, 3)
38
did you call me? batch: 3 shape is: (

In [27]:
X.shape

(20, 1, 128, 128, 3)

In [28]:
y.shape

(20, 5)

In [29]:
y

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]])

In [30]:
cls.get_randomized_borderlabels_and_batches_per_video().iloc[-1]['total_batches']

174

In [31]:
cls.get_randomized_borderlabels_and_batches_per_video(batch_size=20, training=True)

,videoID,batches_in_video,random_order,total_batches
0,2,38,1,38
1,4,19,2,57
2,6,16,3,73
3,3,23,4,96
4,1,24,5,120
5,5,20,6,140


In [32]:
len(gen)

len called len is: 140


140

In [33]:
len(gen)

140

In [34]:
cls.get_borderlabels_batch(videoID=1, batch_nr=23, batch_size=20)

,frameNr,label
0,460,0
1,461,0
2,462,0
3,463,0
4,464,0
5,465,1
6,466,2
7,467,2
8,468,2
9,469,2


In [35]:
import numpy as np

In [21]:
y = np.where(y == 9, 3, y)
y = np.where(y == 8, 4, y)

In [22]:
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])